In [14]:
import pandas as pd
import matplotlib
import plotly
import tkinter
import os
import numpy as np
import seaborn
import plotly.express


In [15]:
with open('teams_path') as f:
    lines = f.readlines()
teams_path = lines[0] + '/'
print(teams_path)

C:\Teams\Hoogheemraadschap van Delfland\Programma DigiTrans - Zoutindringing/


In [16]:
save_TS_plots = True
save_violin_plots = True
save_quantile_plots = False
locatie = 'westland'

In [17]:

df = pd.read_table(teams_path + 'telecontrol/'+locatie+'.csv',delimiter = '\t')
numeric_cols = df.columns[2:]
df[numeric_cols] = df[numeric_cols].stack().str.replace(',','.').unstack()
df.head()
df.drop(df.tail(5).index,inplace = True)
df['datetime'] = pd.to_datetime(df['Datum'] + ' ' + df['Tijd (Europe/Amsterdam)'])
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

#df['OW000-008/WNS7670 - gemeten waarde [mS/cm]'] = pd.to_numeric(df['OW000-008/WNS7670 - gemeten waarde [mS/cm]'],errors='coerce')
df.head()
df = df.set_index('Datum')
df[numeric_cols] = np.where(df[numeric_cols]>=100 , np.nan, df[numeric_cols])
df['maand'] = df['datetime'].dt.strftime('%b')


In [18]:
%%capture
isExist = os.path.exists(teams_path+'plots/' + locatie)
if not isExist:
  os.makedirs(teams_path+'plots/' + locatie)


if save_TS_plots:
  root = tkinter.Tk()
  my_dpi = root.winfo_fpixels('1i')

  subsets =  (['2015-01-01','2015-31-12'], ['2016-01-01','2016-31-12'], ['2017-01-01','2017-31-12'], ['2018-01-01','2018-31-12'],
              ['2019-01-01','2019-31-12'], ['2020-01-01','2020-31-12'], ['2021-01-01','2021-31-12'], ['2022-01-01','2022-31-12'],
              ['2015-01-01','2022-31-12']
              )


  for subset in subsets:
  #subset = ['2015-01-01','2015-31-12']
      plot1 = df.loc[subset[0]:subset[1]].plot('datetime', df.columns[2],figsize=(1920/my_dpi, 1080/my_dpi))
      axes= plot1.xaxis
      axes.set_major_locator(matplotlib.dates.MonthLocator(interval=1))
      axes.get_major_ticks()[-1].label1.set_visible(False)
      plot1.set_xlabel('Datum')
      plot1.set_ylabel('EGV')
      plot1.set_title('Van ' + subset[0] + ' t/m ' + subset[1] + '\n Locatie: ' + locatie)
      plot = plot1.figure
      plot_int = plotly.tools.mpl_to_plotly(plot)
      plot_int.write_html(teams_path+"plots/" + locatie + '/' + locatie + subset[0] + '_' + subset[1] + ".html")


if save_violin_plots:
  plot2 = plotly.express.violin(df,x = 'maand', y = numeric_cols[1],points='all',title='EGV verdeling per maand op locatie: ' + locatie, labels= {numeric_cols[1] : 'EGV (mS/cm)'},
  category_orders={"maand": ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]})
  #plot2.set_ylabel('EGV (mS/cm')
  #plot2.show()
  #plot2 = plot2.figure
  #plot_int = plotly.tools.mpl_to_plotly(plot2)
  plot2.write_html(teams_path+"plots/" + locatie + '/' + locatie + "_violin.html")

#if save_quantile_plots:
  

c:\Users\ds_sboeschoten\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\matplotlylib\renderer.py:611: UserWarning:

I found a path object that I don't think is part of a bar chart. Ignoring.

c:\Users\ds_sboeschoten\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\matplotlylib\renderer.py:611: UserWarning:

I found a path object that I don't think is part of a bar chart. Ignoring.

c:\Users\ds_sboeschoten\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\matplotlylib\renderer.py:611: UserWarning:

I found a path object that I don't think is part of a bar chart. Ignoring.

c:\Users\ds_sboeschoten\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\matplotlylib\renderer.py:611: UserWarning:

I found a path object that I don't think is part of a bar chart. Ignoring.

c:\Users\ds_sboeschoten\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\matplotlylib\renderer.py:611: UserWarning:

I found a path object that I do

In [19]:

df.groupby('maand').quantile([0.10,0.20,0.30,0.40,0.50,0.60,0.70,0.80,0.90])

C:\Users\ds_sboeschoten\AppData\Local\Temp\3\ipykernel_7676\3088433796.py:1: FutureWarning:

Dropping invalid columns in DataFrameGroupBy.quantile is deprecated. In a future version, a TypeError will be raised. Before calling .quantile, select only columns which should be valid for the function.



OW000-002/WNS3838 - gemeten waarde [°C]  \
maand                                                
Apr   0.1                                      8.4   
      0.2                                      8.9   
      0.3                                      9.4   
      0.4                                      9.8   
      0.5                                     10.3   
...                                            ...   
Sep   0.5                                     19.5   
      0.6                                     19.8   
      0.7                                     20.7   
      0.8                                     21.3   
      0.9                                     22.8   

           OW000-002/WNS7670 - gemeten waarde [mS/cm]            datetime  
maand                                                                      
Apr   0.1                                      1.8365 2017-04-19 18:18:12  
      0.2                                      5.4310 2018-04-05 07:32:12  
      0.3                                      8.9005 2018-04-22 16:31:24  
      0.4                                     12.1600 2019-04-08 00:35:12  
      0.5                                     14.8765 2019-04-26 21:34:00  
...                                               ...                 ...  
Sep   0.5                                     19.4755 2018-09-16 01:05:00  
      0.6                                     21.9036 2018-09-30 19:16:00  
      0.7                                     24.2580 2019-09-15 09:05:00  
      0.8                                     26.9456 2019-09-29 12:32:00  
      0.9                                     30.2320 2021-09-08 09:01:00  

[108 rows x 3 columns]